In [66]:
import numpy as np
import pandas as pd
#from sklearn.cluster import KMeans
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial

In [67]:
data = pd.read_csv('data.csv')
labels = pd.read_csv('label.csv',names=['label'],header=None)

In [68]:
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
data.count()

0        9999
0.1      9999
0.2      9999
0.3      9999
0.4      9999
         ... 
0.663    9999
0.664    9999
0.665    9999
0.666    9999
0.667    9999
Length: 784, dtype: int64

In [70]:
#Split data
from sklearn.model_selection import train_test_split
train_data, txt_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

In [71]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
            # np.sum()
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Initialize_Centroids(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        #Mode = 2 => Jaccard
        #Mode = 3 => Cosine
        centroid_value_dict = self.Initialize_Centroids(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict

In [72]:
def predict_cluster_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [73]:
def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

In [74]:
def accuracy(centroids, centroid_Labels, txt_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(txt_data.shape[0]):
        featureset = txt_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = txt_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [75]:
Model_A = KMeans()
centroids1,clusters1 = Model_A.train_Kmeans(data,10, max_iter=100,mode=1)

Tolerance for the Iteration  0 :  25008.10947368421
Tolerance for the Iteration  1 :  5478.158924741103
Tolerance for the Iteration  2 :  3872.870050737668
Tolerance for the Iteration  3 :  3517.0231585518104
Tolerance for the Iteration  4 :  4550.081517644161
Tolerance for the Iteration  5 :  3377.3061325383746
Tolerance for the Iteration  6 :  1794.630486593843
Tolerance for the Iteration  7 :  1153.5961500115493
Tolerance for the Iteration  8 :  1004.7614314115307
Tolerance for the Iteration  9 :  761.9693482937989
Tolerance for the Iteration  10 :  691.3359157361575
Tolerance for the Iteration  11 :  734.2117453115412
Tolerance for the Iteration  12 :  745.1058592570221
Tolerance for the Iteration  13 :  691.8065674163232
Tolerance for the Iteration  14 :  863.8808783317359
Tolerance for the Iteration  15 :  1033.7545931129425
Tolerance for the Iteration  16 :  852.7893730047883
Tolerance for the Iteration  17 :  830.7619762569832
Tolerance for the Iteration  18 :  594.183409314363

In [76]:
Euclidean_SSE = Model_A.calculate_SSE(centroids1,clusters1,data)

In [77]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25473485446.671112


In [78]:
cluster_lab_a = predict_cluster_labels(centroids1,clusters1,labels)
cluster_lab_a

array([4, 0, 7, 2, 9, 3, 3, 0, 8, 1])

In [79]:
Acc_Euclidean = accuracy(centroids1, cluster_lab_a,txt_data,test_labels)
Acc_Euclidean

0.06625

In [80]:
Model_B = KMeans()
centroids2,clusters2 = Model_B.train_Kmeans(data,10, max_iter=100,mode=2)
Jaccard_SSE = Model_B.calculate_SSE(centroids2,clusters2,data)

Tolerance for the Iteration  0 :  36246.78698525674
Tolerance for the Iteration  1 :  10608.905063968514
Tolerance for the Iteration  2 :  6219.683158687381
Tolerance for the Iteration  3 :  2991.0590846943596
Tolerance for the Iteration  4 :  2339.4272570114153
Tolerance for the Iteration  5 :  2641.248626014233
Tolerance for the Iteration  6 :  4745.359959909035
Tolerance for the Iteration  7 :  1723.4542732505656
Tolerance for the Iteration  8 :  1443.5303115121637
Tolerance for the Iteration  9 :  781.7453669001505
Tolerance for the Iteration  10 :  1510.1749679315758
Tolerance for the Iteration  11 :  1308.3374630613744
Tolerance for the Iteration  12 :  1970.6197220128724
Tolerance for the Iteration  13 :  0.0


In [81]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34361687572.938736


In [82]:
cluster_lab_b = predict_cluster_labels(centroids2,clusters2,labels)
cluster_lab_b

array([1, 0, 8, 2, 6, 3, 1, 1, 1, 5])

In [83]:
Acc_Jaccard = accuracy(centroids2, cluster_lab_b,txt_data,test_labels)
Acc_Jaccard

0.115

In [84]:
Model_C = KMeans()
centroids3,clusters3 = Model_C.train_Kmeans(data,10, max_iter = 100,mode=3)

Tolerance for the Iteration  0 :  30780.160864345737
Tolerance for the Iteration  1 :  4764.364272626137
Tolerance for the Iteration  2 :  2423.487369653711
Tolerance for the Iteration  3 :  2295.17617708124
Tolerance for the Iteration  4 :  1904.420332389078
Tolerance for the Iteration  5 :  1436.248691145808
Tolerance for the Iteration  6 :  1257.9087995920036
Tolerance for the Iteration  7 :  1225.5769995531723
Tolerance for the Iteration  8 :  893.7499129974178
Tolerance for the Iteration  9 :  1220.2700974459194
Tolerance for the Iteration  10 :  1615.9089957702877
Tolerance for the Iteration  11 :  1651.75835863216
Tolerance for the Iteration  12 :  1716.7813104701913
Tolerance for the Iteration  13 :  1527.0845556514237
Tolerance for the Iteration  14 :  1164.6939703977582
Tolerance for the Iteration  15 :  636.1432678528433
Tolerance for the Iteration  16 :  495.8264869399854
Tolerance for the Iteration  17 :  522.3062313084466
Tolerance for the Iteration  18 :  356.42337187090

In [85]:
Cosine_SSE = Model_C.calculate_SSE(centroids3,clusters3,data)

In [86]:
print("Euclidean_SSE:",Euclidean_SSE)
print("Jacard_SSE:",Jaccard_SSE)
print("Cosine_SSE :",Cosine_SSE)

Euclidean_SSE: 25473485446.671112
Jacard_SSE: 34361687572.938736
Cosine_SSE : 25599732295.95894


In [87]:
cluster_labels_c = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels_c

array([3, 0, 1, 7, 2, 0, 9, 6, 3, 4])

In [88]:
Acc_Cosine = accuracy(centroids3, cluster_labels_c,txt_data,test_labels)

In [89]:
Acc_Euclidean
Acc_Jaccard
Acc_Cosine

0.085

In [90]:
print("Euclidean_acc:",Acc_Euclidean)
print("Jacard_acc:",Acc_Jaccard)
print("Cosine_acc :",Acc_Cosine)

Euclidean_acc: 0.06625
Jacard_acc: 0.115
Cosine_acc : 0.085
